In [1]:
# Install needed libraries (run this cell first!)
!pip install --quiet yt-dlp ffmpeg-python torch torchaudio transformers streamlit speechbrain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import subprocess
import torchaudio
import torch
from speechbrain.pretrained import EncoderClassifier
import yt_dlp


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
<ipython-input-5-0f12fb55d26c>:5: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


In [2]:
# Paste your video URL here (YouTube or direct MP4 link)
VIDEO_URL = "https://youtu.be/DDjWTWHHkpk?si=oIj6Fuy8Hg2E8U_l"  # Example: Replace with your actual link!


In [6]:
def download_video(url, out_path="input_video.mp4"):
    """
    Downloads a video from YouTube or direct MP4 link.
    Returns the filename of the downloaded video.
    """
    # If it's a YouTube link, use yt-dlp
    if "youtube.com" in url or "youtu.be" in url:
        ydl_opts = {'outtmpl': out_path}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    else:
        # For direct links, use wget/curl fallback
        os.system(f"wget -O {out_path} {url}")
    return out_path

video_file = download_video(VIDEO_URL)
print(f"Downloaded video: {video_file}")


[youtube] Extracting URL: https://youtu.be/DDjWTWHHkpk?si=oIj6Fuy8Hg2E8U_l
[youtube] DDjWTWHHkpk: Downloading webpage
[youtube] DDjWTWHHkpk: Downloading tv client config
[youtube] DDjWTWHHkpk: Downloading player f203bbc8-main
[youtube] DDjWTWHHkpk: Downloading tv player API JSON
[youtube] DDjWTWHHkpk: Downloading ios player API JSON
[youtube] DDjWTWHHkpk: Downloading m3u8 information
[info] DDjWTWHHkpk: Downloading 1 format(s): 399+251
[download] Destination: input_video.mp4.f399.mp4
[download] 100% of   62.45MiB in 00:00:01 at 31.78MiB/s  
[download] Destination: input_video.mp4.f251.webm
[download] 100% of    4.57MiB in 00:00:00 at 9.99MiB/s   
[Merger] Merging formats into "input_video.mp4.webm"
Deleting original file input_video.mp4.f399.mp4 (pass -k to keep)
Deleting original file input_video.mp4.f251.webm (pass -k to keep)
Downloaded video: input_video.mp4


In [7]:
def extract_audio(video_path, audio_path="audio.wav"):
    """
    Extracts audio from a video file using ffmpeg.
    Returns the filename of the audio file.
    """
    # Remove if already exists
    if os.path.exists(audio_path):
        os.remove(audio_path)
    # Extract audio with ffmpeg
    cmd = f"ffmpeg -y -i {video_path} -ar 16000 -ac 1 -vn {audio_path}"
    subprocess.call(cmd, shell=True)
    return audio_path

audio_file = extract_audio(video_file)
print(f"Extracted audio file: {audio_file}")


Extracted audio file: audio.wav


In [8]:
def extract_audio(video_path, audio_path="/content/audio.wav"):
    """
    Extracts audio from a video file using ffmpeg.
    Returns the filename of the audio file.
    """
    # Remove if already exists
    if os.path.exists(audio_path):
        os.remove(audio_path)
    # Extract audio with ffmpeg
    cmd = f"ffmpeg -y -i {video_path} -ar 16000 -ac 1 -vn {audio_path}"
    # Use subprocess.run to capture output and check the return code
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"FFmpeg command failed with error code {result.returncode}")
        print("FFmpeg stderr:")
        print(result.stderr)
        # Optionally, raise an error or exit if audio extraction fails
        raise RuntimeError(f"Failed to extract audio using FFmpeg. See stderr above.")
    else:
        print("FFmpeg stdout:")
        print(result.stdout)
        print("FFmpeg stderr:")
        print(result.stderr) # ffmpeg often outputs info/warnings to stderr

    # Check if the audio file was actually created
    if not os.path.exists(audio_path):
         raise FileNotFoundError(f"Audio file '{audio_path}' was not created after FFmpeg execution.")

    return audio_path

In [9]:
# Download the pre-trained English accent classifier (SpeechBrain)
accent_model = EncoderClassifier.from_hparams(
    source="speechbrain/lang-id-commonlanguage_ecapa",
    savedir="tmp_accent_model"
)


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/lang-id-commonlanguage_ecapa' if not cached


hyperparams.yaml:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--lang-id-commonlanguage_ecapa/snapshots/9d809a7e1fdf48afa53e1b41ee8ae0af9ce46308/hyperparams.yaml' -> '/content/tmp_accent_model/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/lang-id-commonlanguage_ecapa' if not cached
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook f

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--lang-id-commonlanguage_ecapa/snapshots/9d809a7e1fdf48afa53e1b41ee8ae0af9ce46308/embedding_model.ckpt' -> '/content/tmp_accent_model/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /content/tmp_accent_model/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/lang-id-commonlanguage_ecapa' if not cached


classifier.ckpt:   0%|          | 0.00/35.4k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--lang-id-commonlanguage_ecapa/snapshots/9d809a7e1fdf48afa53e1b41ee8ae0af9ce46308/classifier.ckpt' -> '/content/tmp_accent_model/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /content/tmp_accent_model/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/lang-id-commonlanguage_ecapa' if not cached


label_encoder.txt:   0%|          | 0.00/788 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--lang-id-commonlanguage_ecapa/snapshots/9d809a7e1fdf48afa53e1b41ee8ae0af9ce46308/label_encoder.txt' -> '/content/tmp_accent_model/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /content/tmp_accent_model/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /content/tmp_accent_model/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /content/tmp_accent_model/classifier.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): label_encoder -> /content/tmp_accent_model/label_encoder.ckpt
DEBUG:speechbrain.dataio.encoder:Loaded cate

Used to Debuging the code

In [10]:
# List the files to see if input_video.mp4 is present
import os
print(os.listdir('.'))


['.config', 'tmp_accent_model', 'input_video.mp4.webm', 'sample_data']


TO check the debug file path

In [11]:
# Try extracting audio again, but print output to check for errors
video_path = "/content/input_video.mp4.webm"  # or whatever your filename is!
audio_path = "audio.wav"

os.system(f"ffmpeg -y -i {video_path} -ar 16000 -ac 1 -vn {audio_path}")

# See if audio.wav was created
print(os.listdir('.'))


['.config', 'tmp_accent_model', 'input_video.mp4.webm', 'audio.wav', 'sample_data']


Check the Size of the file

In [12]:
# Check if the file now exists and get its size
import os
print("audio.wav exists:", os.path.exists(audio_path))
if os.path.exists(audio_path):
    print("audio.wav size (bytes):", os.path.getsize(audio_path))


audio.wav exists: True
audio.wav size (bytes): 9308494


In [13]:
# Load the audio file (must be 16kHz mono)
signal, fs = torchaudio.load(audio_file)

# If stereo, take only the first channel
if signal.shape[0] > 1:
    signal = signal[0].unsqueeze(0)

# Run classification
prediction = accent_model.classify_batch(signal)
pred_label = prediction[3][0]
pred_scores = prediction[1][0]

# Convert score to percentage
confidence = float(pred_scores.max()) * 100

# Display top label and score
print(f"Predicted Accent: {pred_label}")
print(f"Confidence: {confidence:.1f}%")
print("Possible accent labels:", accent_model.hparams.label_encoder.lab2ind.keys())



Predicted Accent: English
Confidence: 66.8%
Possible accent labels: dict_keys(['Basque', 'Romansh_Sursilvan', 'Sakha', 'Georgian', 'Greek', 'Hakha_Chin', 'Ukrainian', 'Interlingua', 'Persian', 'Polish', 'Dutch', 'Chinese_Hongkong', 'Japanese', 'Portuguese', 'Italian', 'Catalan', 'Chuvash', 'Swedish', 'Spanish', 'Slovenian', 'Tamil', 'Breton', 'Russian', 'Czech', 'English', 'French', 'Tatar', 'Welsh', 'Kyrgyz', 'Esperanto', 'Kinyarwanda', 'Dhivehi', 'Turkish', 'Latvian', 'Estonian', 'Arabic', 'Frisian', 'Mongolian', 'Chinese_Taiwan', 'Indonesian', 'Maltese', 'Kabyle', 'Romanian', 'Chinese_China', 'German'])


In [14]:
explanation = f"The speaker's English accent was classified as '{pred_label}' with a confidence score of {confidence:.1f}%. This means the model is {confidence:.0f}% sure the person sounds most similar to this accent group."

print(explanation)


The speaker's English accent was classified as 'English' with a confidence score of 66.8%. This means the model is 67% sure the person sounds most similar to this accent group.


In [15]:
# Save as app.py in Colab for launching a simple web UI
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import os
import subprocess
import torchaudio
from speechbrain.pretrained import EncoderClassifier

st.title("🗣️ English Accent Classifier (Proof of Concept)")

url = st.text_input("Enter public video URL (YouTube or direct MP4):")
if st.button("Analyze"):
    with st.spinner("Downloading video..."):
        if "youtube.com" in url or "youtu.be" in url:
            os.system(f'yt-dlp -o input_video.mp4 "{url}"')
        else:
            os.system(f'wget -O input_video.mp4 "{url}"')
    with st.spinner("Extracting audio..."):
        os.system("ffmpeg -y -i input_video.mp4 -ar 16000 -ac 1 -vn audio.wav")
    with st.spinner("Classifying accent..."):
        accent_model = EncoderClassifier.from_hparams(
            source="speechbrain/lang-id-commonlanguage_ecapa",
            savedir="tmp_accent_model"
        )
        signal, fs = torchaudio.load("audio.wav")
        if signal.shape[0] > 1:
            signal = signal[0].unsqueeze(0)
        prediction = accent_model.classify_batch(signal)
        pred_label = prediction[3][0]
        pred_scores = prediction[1][0]
        confidence = float(pred_scores.max()) * 100
        st.success(f"Predicted Accent: {pred_label} ({confidence:.1f}%)")
        st.info(f"The model is {confidence:.0f}% confident this is a {pred_label} English accent.")
''')

print("Streamlit app code saved as app.py!")
print("To launch the UI, run: !streamlit run app.py --server.headless true --server.port 8501")


Streamlit app code saved as app.py!
To launch the UI, run: !streamlit run app.py --server.headless true --server.port 8501
